In [32]:
import os
import json
import kagglehub
import pandas as pd
import numpy as np

In [3]:
# Download latest-version
path = kagglehub.dataset_download("promptcloud/careerbuilder-job-listing-2020")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\edan\.cache\kagglehub\datasets\promptcloud\careerbuilder-job-listing-2020\versions\1


In [4]:
file = os.listdir(path)
print(file)
file_path = os.path.join(path ,file[0])

['marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson']


In [5]:
load_rows = []
with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        try:
            load_rows.append(json.loads(line))
        except Exception as e:
            print(f"Error on line {i+1}: {e}")

# Read in to dataframe
df = pd.DataFrame(load_rows)
df.head()

Error on line 6614: Extra data: line 1 column 7004 (char 7003)


,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,country,post_date,job_description,...,inferred_state,inferred_country,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,is_consumed_job,job_requirements,contact_email,test_contact_email
0,9a457ef257fecf231693a6ba08f50293,2020-06-26 01:54:03 +0000,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,Asphalt/Concrete Senior Project Manager,GPAC,Houston,TX,US,2020-06-25,SR. PROJECT MANAGER WANTED!!! My client is a M...,...,Texas,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ba471e2faf6f79caf22cddebbedbc0e8,2020-05-17 01:21:05 +0000,https://www.careerbuilder.com/job/J3P0FL6X795R...,Amazon Warehouse Team - Full Time,Amazon Fulfillment,Cincinnati,OH,US,2020-05-16,"Shifts: Overnight, Early Morning, Day, Evening...",...,Ohio,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6f00bd02d63c633b5af453366f25c21e,2020-06-27 04:53:42 +0000,https://www.careerbuilder.com/job/J3W3XK69NHHR...,Amazon Warehouse Associate - Morning Shifts Av...,Amazon Fulfillment,Peabody,MA,US,2020-06-26,"Shifts: Early Morning, Day, Evening Location: ...",...,Massachusetts,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,US,2020-06-02,Do you pride yourself on attention to detail a...,...,Minnesota,United states,10,14.00,14.00,$,NaN,NaN,NaN,NaN
4,31753dc342a1b2a07db712454c0d5f87,2020-05-23 01:19:07 +0000,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,Graphics Designer,The North West Company - U.S.,Anchorage,AK,US,2020-05-22,In your role as Graphics Designer for Alaska C...,...,Alaska,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.columns

Index(['uniq_id', 'crawl_timestamp', 'url', 'job_title', 'company_name',
       'city', 'state', 'country', 'post_date', 'job_description', 'job_type',
       'inferred_salary_time_unit', 'company_description', 'salary_offered',
       'job_board', 'geo', 'job_post_lang', 'valid_through',
       'html_job_description', 'inferred_iso2_lang_code',
       'inferred_iso3_lang_code', 'site_name', 'domain', 'postdate_yyyymmdd',
       'has_expired', 'last_expiry_check_date', 'latest_expiry_check_date',
       'duplicate_status', 'postdate_in_indexname_format', 'inferred_city',
       'inferred_state', 'inferred_country', 'fitness_score',
       'inferred_salary_from', 'inferred_salary_to',
       'inferred_salary_currency', 'is_consumed_job', 'job_requirements',
       'contact_email', 'test_contact_email'],
      dtype='object')

## Data preporcessing for ML model

In [6]:
# Check for count of datatypes of values for each column
mixed_type_columns = []
for col in df.columns:
    type_counts = df[col].map(type).value_counts()
    # print(f"Column: {col} - Type Counts: {type_counts}")
    if len(type_counts) > 1:
        mixed_type_columns.append({
            'column': col,
            'types': {t.__name__: int(c) for t, c in type_counts.items()}
        })
mixed_type_columns

[{'column': 'country', 'types': {'str': 29973, 'float': 9}},
 {'column': 'job_type', 'types': {'str': 29750, 'float': 232}},
 {'column': 'company_description', 'types': {'str': 15839, 'float': 14143}},
 {'column': 'valid_through', 'types': {'str': 29973, 'float': 9}},
 {'column': 'inferred_salary_from', 'types': {'float': 25791, 'str': 4191}},
 {'column': 'inferred_salary_to', 'types': {'float': 25791, 'str': 4191}},
 {'column': 'inferred_salary_currency',
  'types': {'float': 26464, 'str': 3518}},
 {'column': 'is_consumed_job', 'types': {'float': 29767, 'str': 215}},
 {'column': 'job_requirements', 'types': {'float': 29979, 'str': 3}},
 {'column': 'contact_email', 'types': {'float': 29970, 'str': 12}},
 {'column': 'test_contact_email',
  'types': {'float': 29978, 'str': 3, 'list': 1}}]

In [7]:
# Remove row with list
career_data = df.copy()
career_data = career_data[~career_data["test_contact_email"].apply(lambda x: isinstance(x, list))]
career_data

,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,country,post_date,job_description,...,inferred_state,inferred_country,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,is_consumed_job,job_requirements,contact_email,test_contact_email
0,9a457ef257fecf231693a6ba08f50293,2020-06-26 01:54:03 +0000,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,Asphalt/Concrete Senior Project Manager,GPAC,Houston,TX,US,2020-06-25,SR. PROJECT MANAGER WANTED!!! My client is a M...,...,Texas,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ba471e2faf6f79caf22cddebbedbc0e8,2020-05-17 01:21:05 +0000,https://www.careerbuilder.com/job/J3P0FL6X795R...,Amazon Warehouse Team - Full Time,Amazon Fulfillment,Cincinnati,OH,US,2020-05-16,"Shifts: Overnight, Early Morning, Day, Evening...",...,Ohio,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6f00bd02d63c633b5af453366f25c21e,2020-06-27 04:53:42 +0000,https://www.careerbuilder.com/job/J3W3XK69NHHR...,Amazon Warehouse Associate - Morning Shifts Av...,Amazon Fulfillment,Peabody,MA,US,2020-06-26,"Shifts: Early Morning, Day, Evening Location: ...",...,Massachusetts,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,US,2020-06-02,Do you pride yourself on attention to detail a...,...,Minnesota,United states,10,14.00,14.00,$,NaN,NaN,NaN,NaN
4,31753dc342a1b2a07db712454c0d5f87,2020-05-23 01:19:07 +0000,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,Graphics Designer,The North West Company - U.S.,Anchorage,AK,US,2020-05-22,In your role as Graphics Designer for Alaska C...,...,Alaska,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29977,936736b277678e921106037c4dd7a84c,2020-06-09 00:58:51 +0000,https://www.careerbuilder.com/job/J3T0565YGTCC...,Part Time OR Temporary Warehouse Associate,Warehouse,Wilmer,TX,US,2020-06-08,"Capstone Logistics, LLC is part of The Transpo...",...,Texas,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29978,149876538eeb2fb04d197edea7cbea27,2020-06-05 01:59:14 +0000,https://www.careerbuilder.com/job/J3W22P7514NR...,Patient Access Specialist,Robert Half,Birmingham,AL,US,2020-06-04,Ref ID: 00100 [ Phone number blocked ] Classif...,...,Alabama,United states,10,12.50,14.00,$,NaN,NaN,NaN,NaN
29979,7874ba31564aa0ef58fcbb665f6e0b3d,2020-06-15 01:12:52 +0000,https://www.careerbuilder.com/job/J3Q6WX61BPK9...,Registered Nurse - ICU - 13 Week Contract,Jackson Nurse Professionals,Roanoke,VA,US,2020-06-14,We’re hiring a highly skilled Registered Nurse...,...,Virginia,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29980,503793ea84532a4b308d92024bef084b,2020-06-02 02:07:17 +0000,https://www.careerbuilder.com/job/J3M1LG6J8GMC...,Amazon Prime Now Shopper - Earn $15/hr or more...,Amazon Fulfillment,Deer Park,WI,US,2020-05-31,sFulfillment Centers – Work inside an Amazon w...,...,Wisconsin,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Check the number of unique values in each column
unique_values = career_data.nunique() # This creates a pandas series
# Make a list of the unique values with only 1 unique value
cols_to_drop = unique_values[unique_values == 1]
cols_to_drop_list = cols_to_drop.index
cols_to_drop_list

Index(['country', 'company_description', 'job_board', 'geo', 'job_post_lang',
       'inferred_iso2_lang_code', 'inferred_iso3_lang_code', 'site_name',
       'domain', 'has_expired', 'duplicate_status', 'inferred_country',
       'is_consumed_job'],
      dtype='object')

### Drop columns that provide no value to the Model - only has one unique value or is majority null

In [9]:
# Remove cols as per the cols_to_drop
career_data = career_data.drop(columns=cols_to_drop_list)
career_data

,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,job_type,...,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,job_requirements,contact_email,test_contact_email
0,9a457ef257fecf231693a6ba08f50293,2020-06-26 01:54:03 +0000,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,Asphalt/Concrete Senior Project Manager,GPAC,Houston,TX,2020-06-25,SR. PROJECT MANAGER WANTED!!! My client is a M...,Full-Time,...,2020.06.22,Houston,Texas,10,NaN,NaN,NaN,NaN,NaN,NaN
1,ba471e2faf6f79caf22cddebbedbc0e8,2020-05-17 01:21:05 +0000,https://www.careerbuilder.com/job/J3P0FL6X795R...,Amazon Warehouse Team - Full Time,Amazon Fulfillment,Cincinnati,OH,2020-05-16,"Shifts: Overnight, Early Morning, Day, Evening...",Full-Time,...,2020.05.11,Cincinnati,Ohio,10,NaN,NaN,NaN,NaN,NaN,NaN
2,6f00bd02d63c633b5af453366f25c21e,2020-06-27 04:53:42 +0000,https://www.careerbuilder.com/job/J3W3XK69NHHR...,Amazon Warehouse Associate - Morning Shifts Av...,Amazon Fulfillment,Peabody,MA,2020-06-26,"Shifts: Early Morning, Day, Evening Location: ...",Full-Time,...,2020.06.22,Peabody,Massachusetts,10,NaN,NaN,NaN,NaN,NaN,NaN
3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,Full-Time,...,2020.06.01,Villard,Minnesota,10,14.00,14.00,$,NaN,NaN,NaN
4,31753dc342a1b2a07db712454c0d5f87,2020-05-23 01:19:07 +0000,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,Graphics Designer,The North West Company - U.S.,Anchorage,AK,2020-05-22,In your role as Graphics Designer for Alaska C...,Full-Time,...,2020.05.22,Anchorage,Alaska,10,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29977,936736b277678e921106037c4dd7a84c,2020-06-09 00:58:51 +0000,https://www.careerbuilder.com/job/J3T0565YGTCC...,Part Time OR Temporary Warehouse Associate,Warehouse,Wilmer,TX,2020-06-08,"Capstone Logistics, LLC is part of The Transpo...",Full-Time,...,2020.06.01,Wilmer,Texas,10,NaN,NaN,NaN,NaN,NaN,NaN
29978,149876538eeb2fb04d197edea7cbea27,2020-06-05 01:59:14 +0000,https://www.careerbuilder.com/job/J3W22P7514NR...,Patient Access Specialist,Robert Half,Birmingham,AL,2020-06-04,Ref ID: 00100 [ Phone number blocked ] Classif...,Seasonal/Temp,...,2020.06.01,Birmingham,Alabama,10,12.50,14.00,$,NaN,NaN,NaN
29979,7874ba31564aa0ef58fcbb665f6e0b3d,2020-06-15 01:12:52 +0000,https://www.careerbuilder.com/job/J3Q6WX61BPK9...,Registered Nurse - ICU - 13 Week Contract,Jackson Nurse Professionals,Roanoke,VA,2020-06-14,We’re hiring a highly skilled Registered Nurse...,Contractor,...,2020.06.11,Roanoke,Virginia,10,NaN,NaN,NaN,NaN,NaN,NaN
29980,503793ea84532a4b308d92024bef084b,2020-06-02 02:07:17 +0000,https://www.careerbuilder.com/job/J3M1LG6J8GMC...,Amazon Prime Now Shopper - Earn $15/hr or more...,Amazon Fulfillment,Deer Park,WI,2020-05-31,sFulfillment Centers – Work inside an Amazon w...,Full-Time,...,2020.05.22,Deer park,Wisconsin,10,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Get columns that have more than 99% null values
null_cols = career_data.columns[career_data.isnull().mean() > 0.99].tolist()

# drop these columns
career_data = career_data.drop(columns=null_cols)
career_data

,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,job_type,...,postdate_yyyymmdd,last_expiry_check_date,latest_expiry_check_date,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency
0,9a457ef257fecf231693a6ba08f50293,2020-06-26 01:54:03 +0000,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,Asphalt/Concrete Senior Project Manager,GPAC,Houston,TX,2020-06-25,SR. PROJECT MANAGER WANTED!!! My client is a M...,Full-Time,...,20200625,2020.06.26,2020-06-26,2020.06.22,Houston,Texas,10,NaN,NaN,NaN
1,ba471e2faf6f79caf22cddebbedbc0e8,2020-05-17 01:21:05 +0000,https://www.careerbuilder.com/job/J3P0FL6X795R...,Amazon Warehouse Team - Full Time,Amazon Fulfillment,Cincinnati,OH,2020-05-16,"Shifts: Overnight, Early Morning, Day, Evening...",Full-Time,...,20200516,2020.05.17,2020-05-17,2020.05.11,Cincinnati,Ohio,10,NaN,NaN,NaN
2,6f00bd02d63c633b5af453366f25c21e,2020-06-27 04:53:42 +0000,https://www.careerbuilder.com/job/J3W3XK69NHHR...,Amazon Warehouse Associate - Morning Shifts Av...,Amazon Fulfillment,Peabody,MA,2020-06-26,"Shifts: Early Morning, Day, Evening Location: ...",Full-Time,...,20200626,2020.06.27,2020-06-27,2020.06.22,Peabody,Massachusetts,10,NaN,NaN,NaN
3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,Full-Time,...,20200602,2020.06.03,2020-06-03,2020.06.01,Villard,Minnesota,10,14.00,14.00,$
4,31753dc342a1b2a07db712454c0d5f87,2020-05-23 01:19:07 +0000,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,Graphics Designer,The North West Company - U.S.,Anchorage,AK,2020-05-22,In your role as Graphics Designer for Alaska C...,Full-Time,...,20200522,2020.05.23,2020-05-23,2020.05.22,Anchorage,Alaska,10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29977,936736b277678e921106037c4dd7a84c,2020-06-09 00:58:51 +0000,https://www.careerbuilder.com/job/J3T0565YGTCC...,Part Time OR Temporary Warehouse Associate,Warehouse,Wilmer,TX,2020-06-08,"Capstone Logistics, LLC is part of The Transpo...",Full-Time,...,20200608,2020.06.09,2020-06-09,2020.06.01,Wilmer,Texas,10,NaN,NaN,NaN
29978,149876538eeb2fb04d197edea7cbea27,2020-06-05 01:59:14 +0000,https://www.careerbuilder.com/job/J3W22P7514NR...,Patient Access Specialist,Robert Half,Birmingham,AL,2020-06-04,Ref ID: 00100 [ Phone number blocked ] Classif...,Seasonal/Temp,...,20200604,2020.06.05,2020-06-05,2020.06.01,Birmingham,Alabama,10,12.50,14.00,$
29979,7874ba31564aa0ef58fcbb665f6e0b3d,2020-06-15 01:12:52 +0000,https://www.careerbuilder.com/job/J3Q6WX61BPK9...,Registered Nurse - ICU - 13 Week Contract,Jackson Nurse Professionals,Roanoke,VA,2020-06-14,We’re hiring a highly skilled Registered Nurse...,Contractor,...,20200614,2020.06.15,2020-06-15,2020.06.11,Roanoke,Virginia,10,NaN,NaN,NaN
29980,503793ea84532a4b308d92024bef084b,2020-06-02 02:07:17 +0000,https://www.careerbuilder.com/job/J3M1LG6J8GMC...,Amazon Prime Now Shopper - Earn $15/hr or more...,Amazon Fulfillment,Deer Park,WI,2020-05-31,sFulfillment Centers – Work inside an Amazon w...,Full-Time,...,20200531,2020.06.02,2020-06-02,2020.05.22,Deer park,Wisconsin,10,NaN,NaN,NaN


### Data Type Cleaning

In [11]:
career_data.dtypes

uniq_id                         object
crawl_timestamp                 object
url                             object
job_title                       object
company_name                    object
city                            object
state                           object
post_date                       object
job_description                 object
job_type                        object
inferred_salary_time_unit       object
salary_offered                  object
valid_through                   object
html_job_description            object
postdate_yyyymmdd               object
last_expiry_check_date          object
latest_expiry_check_date        object
postdate_in_indexname_format    object
inferred_city                   object
inferred_state                  object
fitness_score                    int64
inferred_salary_from            object
inferred_salary_to              object
inferred_salary_currency        object
dtype: object

In [12]:
career_data.inferred_salary_currency.value_counts(dropna=False)

inferred_salary_currency
NaN    26463
$       3516
£          2
Name: count, dtype: int64

In [73]:
# Drop data without currency
career_data_currency = career_data.copy()
career_data_currency = career_data_currency[~career_data_currency["inferred_salary_currency"].isnull()].reset_index()
career_data_currency

,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,postdate_yyyymmdd,last_expiry_check_date,latest_expiry_check_date,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,20200602,2020.06.03,2020-06-03,2020.06.01,Villard,Minnesota,10,14.00,14.00,$
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52 +0000,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,20200518,2020.05.19,2020-05-19,2020.05.11,Hagerstown,Maryland,10,"41,000.00","62,000.00",$
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27 +0000,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,20200606,2020.06.07,2020-06-07,2020.06.01,Allen,Texas,10,12.00,13.50,$
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25 +0000,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,20200618,2020.06.19,2020-06-19,2020.06.11,Longmont,Colorado,10,16.50,16.50,$
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15 +0000,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,20200530,2020.05.31,2020-05-31,2020.05.22,Skokie,Illinois,10,14.00,14.00,$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58 +0000,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,20200511,2020.05.12,2020-05-12,2020.05.11,Grand rapids,Michigan,10,"11,000.00","14,000.00",$
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27 +0000,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,20200615,2020.06.16,2020-06-16,2020.06.11,Baraboo,Wisconsin,10,"50,000.00","70,000.00",$
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44 +0000,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,20200616,2020.06.17,2020-06-17,2020.06.11,Tukwila,Washington,10,"65,000.00","75,000.00",$
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20 +0000,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,20200612,2020.06.13,2020-06-13,2020.06.11,,New hampshire,10,13.00,13.00,$


In [75]:
# Check dtype value counts for salary_from
career_data_currency["inferred_salary_from"].apply(type).value_counts()

# find rows without dot
no_dot  = career_data_currency[~career_data_currency["inferred_salary_from"].str.contains("\.")]
no_dot["inferred_salary_from"].value_counts(dropna=False)

# Remove rows with inferred_salary_from with values such as USD, Average, + and up
career_data_salary = career_data_currency.copy()

In [76]:
career_data_salary = career_data_salary[~career_data_salary["inferred_salary_from"].str.contains("USD|Average|\+|up|Hour|US|K")]
career_data_salary["inferred_salary_from"].value_counts(dropna=False)

inferred_salary_from
9.50         374
50,000.00    219
15.00        193
14.00        152
12.00        122
            ... 
18,201.00      1
16.30          1
11.97          1
33,280.00      1
14.65          1
Name: count, Length: 430, dtype: int64

In [77]:
# career_data_salary = career_data_salary[~career_data_salary["inferred_salary_to"].str.contains("USD|Average|\+|up|Hour|US|K")]

# sal_from_to = ["inferred_salary_from", "inferred_salary_to"]
career_data_salary["inferred_salary_from"]= career_data_salary["inferred_salary_from"].apply(lambda x : x.replace(",", "")).astype(float)
career_data_salary["inferred_salary_to"]= career_data_salary["inferred_salary_to"].apply(lambda x : x.replace(",", "")).astype(float)
career_data_salary

,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,postdate_yyyymmdd,last_expiry_check_date,latest_expiry_check_date,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,20200602,2020.06.03,2020-06-03,2020.06.01,Villard,Minnesota,10,14.0,14.0,$
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52 +0000,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,20200518,2020.05.19,2020-05-19,2020.05.11,Hagerstown,Maryland,10,41000.0,62000.0,$
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27 +0000,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,20200606,2020.06.07,2020-06-07,2020.06.01,Allen,Texas,10,12.0,13.5,$
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25 +0000,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,20200618,2020.06.19,2020-06-19,2020.06.11,Longmont,Colorado,10,16.5,16.5,$
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15 +0000,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,20200530,2020.05.31,2020-05-31,2020.05.22,Skokie,Illinois,10,14.0,14.0,$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58 +0000,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,20200511,2020.05.12,2020-05-12,2020.05.11,Grand rapids,Michigan,10,11000.0,14000.0,$
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27 +0000,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,20200615,2020.06.16,2020-06-16,2020.06.11,Baraboo,Wisconsin,10,50000.0,70000.0,$
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44 +0000,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,20200616,2020.06.17,2020-06-17,2020.06.11,Tukwila,Washington,10,65000.0,75000.0,$
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20 +0000,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,20200612,2020.06.13,2020-06-13,2020.06.11,,New hampshire,10,13.0,13.0,$


In [78]:
career_data_salary.dtypes

index                             int64
uniq_id                          object
crawl_timestamp                  object
url                              object
job_title                        object
company_name                     object
city                             object
state                            object
post_date                        object
job_description                  object
job_type                         object
inferred_salary_time_unit        object
salary_offered                   object
valid_through                    object
html_job_description             object
postdate_yyyymmdd                object
last_expiry_check_date           object
latest_expiry_check_date         object
postdate_in_indexname_format     object
inferred_city                    object
inferred_state                   object
fitness_score                     int64
inferred_salary_from            float64
inferred_salary_to              float64
inferred_salary_currency         object


In [79]:
# Create mean salary column
career_data_salary["mean_salary"] = career_data_salary[["inferred_salary_from","inferred_salary_to"]].mean(axis=1)
career_data_salary

,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,last_expiry_check_date,latest_expiry_check_date,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,mean_salary
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,2020.06.03,2020-06-03,2020.06.01,Villard,Minnesota,10,14.0,14.0,$,14.00
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52 +0000,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,2020.05.19,2020-05-19,2020.05.11,Hagerstown,Maryland,10,41000.0,62000.0,$,51500.00
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27 +0000,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,2020.06.07,2020-06-07,2020.06.01,Allen,Texas,10,12.0,13.5,$,12.75
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25 +0000,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,2020.06.19,2020-06-19,2020.06.11,Longmont,Colorado,10,16.5,16.5,$,16.50
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15 +0000,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,2020.05.31,2020-05-31,2020.05.22,Skokie,Illinois,10,14.0,14.0,$,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58 +0000,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,2020.05.12,2020-05-12,2020.05.11,Grand rapids,Michigan,10,11000.0,14000.0,$,12500.00
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27 +0000,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,2020.06.16,2020-06-16,2020.06.11,Baraboo,Wisconsin,10,50000.0,70000.0,$,60000.00
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44 +0000,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,2020.06.17,2020-06-17,2020.06.11,Tukwila,Washington,10,65000.0,75000.0,$,70000.00
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20 +0000,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,2020.06.13,2020-06-13,2020.06.11,,New hampshire,10,13.0,13.0,$,13.00


In [80]:
## Convert all salary types based on salary time unit
career_data_salary["inferred_salary_time_unit"].value_counts(dropna=False)


inferred_salary_time_unit
/hour              2161
/year              1304
/Hour                23
/$0.44 Per mile       2
/$16.00 hourly        1
/$12.75 Hourly        1
Name: count, dtype: int64

In [81]:
career_data_salary = career_data_salary[career_data_salary["inferred_salary_time_unit"] != "/$0.44 Per mile"]
career_data_salary["inferred_salary_time_unit"].value_counts(dropna=False)


inferred_salary_time_unit
/hour             2161
/year             1304
/Hour               23
/$16.00 hourly       1
/$12.75 Hourly       1
Name: count, dtype: int64

In [82]:
# modification method
def hour_convert(val):
    if any(word in str(val) for word in ["Hourly", "hourly", "Hour"]):
        return '/hour'
    return val
career_data_salary["inferred_salary_time_unit"] = career_data_salary["inferred_salary_time_unit"].apply(hour_convert)
career_data_salary["inferred_salary_time_unit"].value_counts(dropna=False)


C:\Users\edan\AppData\Local\Temp\ipykernel_21820\2176848471.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_data_salary["inferred_salary_time_unit"] = career_data_salary["inferred_salary_time_unit"].apply(hour_convert)


inferred_salary_time_unit
/hour    2186
/year    1304
Name: count, dtype: int64

In [83]:
# Method to convert all mean_salary to annual basis based on time_unit
HOURS_PER_YEAR = 8 * 5 * 52  # 2080

def to_annual(row):
    if 'hour'  in row['inferred_salary_time_unit'].lower():
        return row['mean_salary'] * HOURS_PER_YEAR
    return row['mean_salary']

career_data_salary['annual_salary'] = career_data_salary.apply(to_annual, axis=1)
career_data_salary

C:\Users\edan\AppData\Local\Temp\ipykernel_21820\1517563477.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_data_salary['annual_salary'] = career_data_salary.apply(to_annual, axis=1)


,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,latest_expiry_check_date,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,mean_salary,annual_salary
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,2020-06-03,2020.06.01,Villard,Minnesota,10,14.0,14.0,$,14.00,29120.0
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52 +0000,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,2020-05-19,2020.05.11,Hagerstown,Maryland,10,41000.0,62000.0,$,51500.00,51500.0
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27 +0000,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,2020-06-07,2020.06.01,Allen,Texas,10,12.0,13.5,$,12.75,26520.0
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25 +0000,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,2020-06-19,2020.06.11,Longmont,Colorado,10,16.5,16.5,$,16.50,34320.0
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15 +0000,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,2020-05-31,2020.05.22,Skokie,Illinois,10,14.0,14.0,$,14.00,29120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58 +0000,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,2020-05-12,2020.05.11,Grand rapids,Michigan,10,11000.0,14000.0,$,12500.00,12500.0
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27 +0000,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,2020-06-16,2020.06.11,Baraboo,Wisconsin,10,50000.0,70000.0,$,60000.00,60000.0
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44 +0000,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,2020-06-17,2020.06.11,Tukwila,Washington,10,65000.0,75000.0,$,70000.00,70000.0
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20 +0000,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,2020-06-13,2020.06.11,,New hampshire,10,13.0,13.0,$,13.00,27040.0


### After Data Cleaning is completed- Proceeding to next - Step - Setup ML model

In [84]:
career_data_salary.annual_salary.dtype

dtype('float64')

In [85]:
career_data_salary_dtype_change.dtypes

uniq_id                                      object
crawl_timestamp                 datetime64[ns, UTC]
url                                          object
job_title                                    object
company_name                                 object
city                                         object
state                                        object
post_date                                    object
job_description                              object
job_type                                     object
inferred_salary_time_unit                    object
salary_offered                               object
valid_through                                object
html_job_description                         object
postdate_yyyymmdd                    datetime64[ns]
last_expiry_check_date                       object
latest_expiry_check_date                     object
postdate_in_indexname_format                 object
inferred_city                                object
inferred_sta

In [86]:
# Format crawl_timestamp and post_dat_YYYYmmdd
career_data_salary_dtype_change = career_data_salary.copy()
career_data_salary_dtype_change["crawl_timestamp"] = pd.to_datetime(career_data_salary_dtype_change["crawl_timestamp"])
career_data_salary_dtype_change["postdate_yyyymmdd"] = pd.to_datetime(career_data_salary_dtype_change["postdate_yyyymmdd"])
career_data_salary_dtype_change.dtypes

index                                         int64
uniq_id                                      object
crawl_timestamp                 datetime64[ns, UTC]
url                                          object
job_title                                    object
company_name                                 object
city                                         object
state                                        object
post_date                                    object
job_description                              object
job_type                                     object
inferred_salary_time_unit                    object
salary_offered                               object
valid_through                                object
html_job_description                         object
postdate_yyyymmdd                    datetime64[ns]
last_expiry_check_date                       object
latest_expiry_check_date                     object
postdate_in_indexname_format                 object
inferred_cit

In [87]:
# Checking for percentage of null values in each column
missing_values_list = [np.nan, "", " ", None]

career_data_salary_dtype_change.isin(missing_values_list).mean().sort_values(ascending=False)*100

inferred_city                   4.555874
job_type                        1.919771
valid_through                   0.257880
html_job_description            0.000000
mean_salary                     0.000000
inferred_salary_currency        0.000000
inferred_salary_to              0.000000
inferred_salary_from            0.000000
fitness_score                   0.000000
inferred_state                  0.000000
postdate_in_indexname_format    0.000000
latest_expiry_check_date        0.000000
last_expiry_check_date          0.000000
postdate_yyyymmdd               0.000000
index                           0.000000
uniq_id                         0.000000
salary_offered                  0.000000
inferred_salary_time_unit       0.000000
job_description                 0.000000
post_date                       0.000000
state                           0.000000
city                            0.000000
company_name                    0.000000
job_title                       0.000000
url             

#### Imputation - process of filling in missing values in data with something else so that the data can be better used for analysis or modelling

In [88]:
# Using inferred_state to the missing values in inferred_city
inferred_city_mapping = career_data_salary_dtype_change.groupby(["inferred_state"])["inferred_city"].apply(lambda x: x.value_counts().index[0]).to_dict()
inferred_city_mapping

{'Alabama': 'Birmingham',
 'Alaska': 'Anchorage',
 'Arizona': 'Phoenix',
 'Arkansas': 'Little rock',
 'California': 'Los angeles',
 'Colorado': 'Denver',
 'Connecticut': 'Hartford',
 'Delaware': 'Wilmington',
 'District of columbia': 'Washington',
 'Florida': 'Miami',
 'Georgia': 'Atlanta',
 'Hawaii': 'Honolulu',
 'Idaho': 'Boise',
 'Illinois': 'Chicago',
 'Indiana': 'Indianapolis',
 'Iowa': 'Des moines',
 'Kansas': 'Kansas city',
 'Kentucky': 'Louisville',
 'Louisiana': 'Baton rouge',
 'Maine': 'Scarborough',
 'Maryland': 'Baltimore',
 'Massachusetts': 'Boston',
 'Michigan': 'Grand rapids',
 'Minnesota': 'Minneapolis',
 'Mississippi': 'Jackson',
 'Missouri': 'Kansas city',
 'Montana': 'Billings',
 'Nebraska': 'Omaha',
 'Nevada': 'Las vegas',
 'New hampshire': '',
 'New jersey': '',
 'New mexico': 'Albuquerque',
 'New york': 'New york',
 'North carolina': 'Charlotte',
 'North dakota': 'Fargo',
 'Ohio': 'Columbus',
 'Oklahoma': 'Oklahoma city',
 'Oregon': 'Portland',
 'Pennsylvania': 'P

In [89]:
type(inferred_city_mapping)

dict

In [90]:
career_data_salary_dtype_change["inferred_city"] = career_data_salary_dtype_change["inferred_city"].replace(["", " "], np.nan)
career_data_salary_dtype_change["inferred_city"] = career_data_salary_dtype_change["inferred_city"].fillna(career_data_salary_dtype_change["inferred_state"].map(inferred_city_mapping))
career_data_salary_dtype_change

,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,latest_expiry_check_date,postdate_in_indexname_format,inferred_city,inferred_state,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,mean_salary,annual_salary
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32+00:00,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,2020-06-03,2020.06.01,Villard,Minnesota,10,14.0,14.0,$,14.00,29120.0
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52+00:00,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,2020-05-19,2020.05.11,Hagerstown,Maryland,10,41000.0,62000.0,$,51500.00,51500.0
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27+00:00,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,2020-06-07,2020.06.01,Allen,Texas,10,12.0,13.5,$,12.75,26520.0
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25+00:00,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,2020-06-19,2020.06.11,Longmont,Colorado,10,16.5,16.5,$,16.50,34320.0
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15+00:00,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,2020-05-31,2020.05.22,Skokie,Illinois,10,14.0,14.0,$,14.00,29120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58+00:00,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,2020-05-12,2020.05.11,Grand rapids,Michigan,10,11000.0,14000.0,$,12500.00,12500.0
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27+00:00,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,2020-06-16,2020.06.11,Baraboo,Wisconsin,10,50000.0,70000.0,$,60000.00,60000.0
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44+00:00,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,2020-06-17,2020.06.11,Tukwila,Washington,10,65000.0,75000.0,$,70000.00,70000.0
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20+00:00,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,2020-06-13,2020.06.11,,New hampshire,10,13.0,13.0,$,13.00,27040.0


In [91]:
career_data_salary_dtype_change.isin(missing_values_list).mean().sort_values(ascending=False)*100

job_type                        1.919771
inferred_city                   1.318052
valid_through                   0.257880
html_job_description            0.000000
mean_salary                     0.000000
inferred_salary_currency        0.000000
inferred_salary_to              0.000000
inferred_salary_from            0.000000
fitness_score                   0.000000
inferred_state                  0.000000
postdate_in_indexname_format    0.000000
latest_expiry_check_date        0.000000
last_expiry_check_date          0.000000
postdate_yyyymmdd               0.000000
index                           0.000000
uniq_id                         0.000000
salary_offered                  0.000000
inferred_salary_time_unit       0.000000
job_description                 0.000000
post_date                       0.000000
state                           0.000000
city                            0.000000
company_name                    0.000000
job_title                       0.000000
url             

### ML Target Column

### Data encoding - LabelEncoding - may impose ordinality on the categorical data as in the numbers assigned may also imply a reason for the assigned numerical values

In [92]:
# Encoding categorical variables - using label encoding - not the best approach
from sklearn.preprocessing import LabelEncoder
career_data_encoded = career_data_salary_dtype_change.copy()
label_encoder = LabelEncoder()
career_data_encoded["inferred_state_encoded"] = label_encoder.fit_transform(career_data_encoded["inferred_state"])
career_data_encoded[["inferred_state", "inferred_state_encoded"]]

,inferred_state,inferred_state_encoded
0,Minnesota,23
1,Maryland,20
2,Texas,43
3,Colorado,5
4,Illinois,13
...,...,...
3513,Michigan,22
3514,Wisconsin,49
3515,Washington,47
3516,New hampshire,29


### Data encoding - One Hot Encoding

In [93]:
# Using pandas get dummies method
cdata_onehot = career_data_salary_dtype_change.copy()
cdata_onehot = pd.get_dummies(cdata_onehot, columns=["inferred_state"], prefix="state_", drop_first=True, dtype=int)
cdata_onehot
# Drop first is used to avoid the dummy variable trap that may lead to multicollinearity - basically the variables itself for example 5 states, the 5th can be already inferred to be the dummy variable when one hot coding in the end and we dont want to give sucha predictor to the model.


,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,state__South dakota,state__Tennessee,state__Texas,state__Utah,state__Vermont,state__Virginia,state__Washington,state__West virginia,state__Wisconsin,state__Wyoming
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32+00:00,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,0,0,0,0,0,0,0,0,0,0
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52+00:00,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,0,0,0,0,0,0,0,0,0,0
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27+00:00,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,0,0,1,0,0,0,0,0,0,0
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25+00:00,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,0,0,0,0,0,0,0,0,0,0
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15+00:00,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58+00:00,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,0,0,0,0,0,0,0,0,0,0
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27+00:00,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,0,0,0,0,0,0,0,0,1,0
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44+00:00,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,0,0,0,0,0,0,1,0,0,0
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20+00:00,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,0,0,0,0,0,0,0,0,0,0


### Scaling - MinMaxScaling

In [94]:
from sklearn.preprocessing import MinMaxScaler
date_info = cdata_onehot[["crawl_timestamp", "postdate_yyyymmdd", "post_date", "postdate_in_indexname_format"]]
display(date_info)
print(date_info.dtypes)

,crawl_timestamp,postdate_yyyymmdd,post_date,postdate_in_indexname_format
0,2020-06-03 01:21:32+00:00,2020-06-02,2020-06-02,2020.06.01
1,2020-05-19 00:58:52+00:00,2020-05-18,2020-05-18,2020.05.11
2,2020-06-07 01:22:27+00:00,2020-06-06,2020-06-06,2020.06.01
3,2020-06-19 01:35:25+00:00,2020-06-18,2020-06-18,2020.06.11
4,2020-05-31 01:06:15+00:00,2020-05-30,2020-05-30,2020.05.22
...,...,...,...,...
3513,2020-05-12 01:13:58+00:00,2020-05-11,2020-05-11,2020.05.11
3514,2020-06-16 02:29:27+00:00,2020-06-15,2020-06-15,2020.06.11
3515,2020-06-17 01:56:44+00:00,2020-06-16,2020-06-16,2020.06.11
3516,2020-06-13 01:47:20+00:00,2020-06-12,2020-06-12,2020.06.11


crawl_timestamp                 datetime64[ns, UTC]
postdate_yyyymmdd                    datetime64[ns]
post_date                                    object
postdate_in_indexname_format                 object
dtype: object


In [95]:
# Convert post date to unixtime
cdata_onehot["postdate_unixtime"] = cdata_onehot["postdate_yyyymmdd"].astype(np.int64)// 10**9

In [96]:
cdata_onehot["postdate_unixtime"]

0       1591056000
1       1589760000
2       1591401600
3       1592438400
4       1590796800
           ...    
3513    1589155200
3514    1592179200
3515    1592265600
3516    1591920000
3517    1591228800
Name: postdate_unixtime, Length: 3490, dtype: int64

In [97]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = cdata_onehot.copy()

data_scaled['postdate_unixtime'] = scaler.fit_transform(data_scaled[['postdate_unixtime']])
data_scaled

,index,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,post_date,job_description,...,state__Tennessee,state__Texas,state__Utah,state__Vermont,state__Virginia,state__Washington,state__West virginia,state__Wisconsin,state__Wyoming,postdate_unixtime
0,3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32+00:00,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,2020-06-02,Do you pride yourself on attention to detail a...,...,0,0,0,0,0,0,0,0,0,0.703297
1,12,4df85e1c64a81a700dc6641b50ec39fa,2020-05-19 00:58:52+00:00,https://www.careerbuilder.com/job/JHP0LK6P0KM0...,Customer Service - New Path for Customer Servi...,Bankers Life,Hagerstown,MD,2020-05-18,Most people doing this job in this region of t...,...,0,0,0,0,0,0,0,0,0,0.538462
2,21,5e686967bb7d59ef2b39b6cd106b5bc0,2020-06-07 01:22:27+00:00,https://www.careerbuilder.com/job/J3T0SL6Z89DJ...,Call Center Representative - Remote,Staffmark,Allen,TX,2020-06-06,Are you passionate about cosmetics and all thi...,...,0,1,0,0,0,0,0,0,0,0.747253
3,29,996fffd7cec89e9a53ef43f1f4ae2f16,2020-06-19 01:35:25+00:00,https://www.careerbuilder.com/job/J3M2X26M1LY7...,Site Lead Security Officer,Securitas USA,Longmont,CO,2020-06-18,: As a Site Lead you play a key role in mainta...,...,0,0,0,0,0,0,0,0,0,0.879121
4,40,81302d03affdb1c063242b469237a2b2,2020-05-31 01:06:15+00:00,https://www.careerbuilder.com/job/J3V3ZK63T9WN...,Full-Time Security Guard-$14/hr,Securitas USA,Skokie,IL,2020-05-30,ACCESS CONTROL OFFICERSSecuritas Security Serv...,...,0,0,0,0,0,0,0,0,0,0.670330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,29959,63d2c72e9bdcd0134b72777b15141f1f,2020-05-12 01:13:58+00:00,https://www.careerbuilder.com/job/J3W71W79KBZB...,Positions Available on All Shifts for ASAP Start,Manpower,Grand Rapids,MI,2020-05-11,"Are you in need of a job fast?If so, Manpower ...",...,0,0,0,0,0,0,0,0,0,0.461538
3514,29960,c6f227b1301e5fd3749a060e14955e1a,2020-06-16 02:29:27+00:00,https://www.careerbuilder.com/job/J3Q55371XH03...,Regional Truck Driver - Home All Weekend,West Side Transport,Baraboo,WI,2020-06-15,Strike a fair balance of work and life with Re...,...,0,0,0,0,0,0,0,1,0,0.846154
3515,29963,89538e77e09e5501c1befed78a40458a,2020-06-17 01:56:44+00:00,https://www.careerbuilder.com/job/J3S3FG6PL9LQ...,Entry Level Enrollment Specialist- Virtual Int...,D&S Recruiting LLC,Tukwila,WA,2020-06-16,We value your health just as much as ours and ...,...,0,0,0,0,0,1,0,0,0,0.857143
3516,29968,67b4bc5a30e01aed8dd7e64b1042e669,2020-06-13 01:47:20+00:00,https://www.careerbuilder.com/job/J3N22C6J3DXZ...,Immediate-Hire Day-Shift Warehouse Position (L...,"Integrity Staffing Solutions, Inc",Nottingham,NH,2020-06-12,Date: 6/12/2020 11:35:00 AMIntegrity Staffing ...,...,0,0,0,0,0,0,0,0,0,0.813187


### Train-Test Split

In [98]:
from sklearn.model_selection import train_test_split

In [99]:
X_train, X_test, y_train, y_test = train_test_split(
    data_scaled.drop(columns = "annual_salary"), data_scaled["annual_salary"]
)
# Where x is the independent variables without the target variable and y is the target variable which is dependent